<a href="https://colab.research.google.com/github/AsaBarthMaron/hemibrain_analysis/blob/master/2020_05_28_extrinsic_LN_connectivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
# Connect to google drive, google sheets
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
# Load workbook from google sheets
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1jj_K7kZVF6uP6iRrXO8-kKMEd5pifBq9LVSccmcAQlU/edit#gid=0')

In [0]:
# 